In [30]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [31]:
# --- edit these if needed ---
PATH = r"C:\Users\chand\Desktop\Exp\data_sets\Housing.csv"
FEATURES = ['area', 'bedrooms', 'stories', 'bathrooms']
TARGET = 'price'
OUT_XLSX = r"C:\Users\chand\Desktop\Exp\data_sets\housing_costs.xlsx"

In [32]:
# Load and keep only needed columns; coerce to numeric and drop missing
df = pd.read_csv(PATH)
cols = FEATURES + [TARGET]
d = df[cols].copy()
d = d.apply(pd.to_numeric, errors='coerce').dropna()

In [33]:
# Fit multiple linear regression
X = d[FEATURES].to_numpy()
y = d[TARGET].to_numpy()
model = LinearRegression().fit(X, y)

In [34]:
# Predictions and per-row differences
yhat = model.predict(X)
residual = y - yhat                      # actual - predicted
squared_error = residual**2
half_squared_cost = 0.5 * squared_error  # J_i = 1/2 * (error^2)
abs_error = np.abs(residual)


In [35]:
# Overall metrics
mse = squared_error.mean()
rmse = np.sqrt(mse)
mae = abs_error.mean()
r2 = model.score(X, y)

In [36]:
# Build export DataFrame (features, actual, predicted, and costs)
out = d.copy()
out['predicted_price'] = yhat
out['residual'] = residual
out['abs_error'] = abs_error
out['squared_error'] = squared_error
out['half_squared_cost'] = half_squared_cost  # per-line-item cost

# Write to Excel: line items + summary
with pd.ExcelWriter(OUT_XLSX, engine='openpyxl') as xw:
    out.to_excel(xw, index=False, sheet_name='line_item_costs')
    summary = pd.DataFrame({
        'metric': (['intercept'] + [f'coef_{f}' for f in FEATURES]
                   + ['R2','MAE','MSE','RMSE','J(theta)=1/2·MSE']),
        'value':  [model.intercept_, *model.coef_, r2, mae, mse, rmse, 0.5*mse]
    })
    summary.to_excel(xw, index=False, sheet_name='summary')

print("Saved:", OUT_XLSX)

Saved: C:\Users\chand\Desktop\Exp\data_sets\housing_costs.xlsx
